<a href="https://colab.research.google.com/github/sakkarin31/musicrecomment/blob/main/test_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics.pairwise import cosine_similarity
import joblib
import ast

autoencoder = load_model("autoencoder_model.h5", custom_objects={"mse": "mean_squared_error"})
encoder = load_model("encoder_model.h5")
# โหลดข้อมูล
df = pd.read_csv("tracks_features.csv")

# เลือกเฉพาะ Feature ที่ใช้เทรนโมเดล
features = ['danceability', 'energy', 'key', 'loudness', 'speechiness',
            'acousticness', 'instrumentalness', 'valence', 'tempo']

# ลบค่าที่เป็น NaN
df_clean = df.dropna(subset=features)

# แปลงข้อมูลเป็น numpy array
X = df_clean[features].values

# Normalize ข้อมูล
scaler = joblib.load("scaler.pkl")
X_scaled = scaler.transform(X)

# ใช้ encoder แปลงเพลงเป็นเวกเตอร์
X_encoded = encoder.predict(X_scaled)
# เพิ่มคอลัมน์ 'vector' ลงใน DataFrame
df_clean["vector"] = list(X_encoded)

# ตรวจสอบว่าเรามีคอลัมน์ 'vector' แล้ว
print(df_clean.head())

def recommend_similar(song_name, artist_name, df, top_n=5):
    # ค้นหาเพลงที่ชื่อเพลงตรงกัน และมีชื่อศิลปินอยู่ในลิสต์ artists
    song = df[(df["name"].str.lower() == song_name.lower()) &
              (df["artists"].apply(lambda x: artist_name.lower() in [a.lower() for a in eval(x)]))]

    if song.empty:
        return "ไม่พบเพลงนี้ใน Dataset"

    # ดึงเวกเตอร์ของเพลงที่ค้นหา
    song_vector = np.array(song["vector"].iloc[0]).reshape(1, -1)

    similarity = cosine_similarity(song_vector, np.stack(df["vector"]))

    indices = similarity.argsort()[0][::-1]

    recommended_songs = df.iloc[indices]

    # กรองเพลงที่เป็นเพลงต้นฉบับออก
    recommended_songs = recommended_songs[~((recommended_songs["name"].str.lower() == song_name.lower()) &
                                            (recommended_songs["artists"].apply(lambda x: artist_name.lower() in [a.lower() for a in eval(x)])))]

    # ลบเพลงซ้ำที่มีชื่อและศิลปินเหมือนกัน
    recommended_songs = recommended_songs.drop_duplicates(subset=["name", "artists"])

    recommended_songs = recommended_songs.head(top_n)

    return recommended_songs[["name", "artists", "album"]]

# ทดสอบแนะนำเพลง
recommend_similar("Will He", "Joji", df_clean)


37626/37626 ━━━━━━━━━━━━━━━━━━━━ 46s 1ms/step
                       id                   name                      album  \
0  7lmeHLHBe4nmXzuXc0HDjk                Testify  The Battle Of Los Angeles   
1  1wsRitfRRtWyEapl0q22o8        Guerrilla Radio  The Battle Of Los Angeles   
2  1hR0fIFK2qRG3f3RF70pb7       Calm Like a Bomb  The Battle Of Los Angeles   
3  2lbASgTSoDO7MTuLAXlTW0              Mic Check  The Battle Of Los Angeles   
4  1MQTmpYOZ6fcMQc56Hdo7T  Sleep Now In the Fire  The Battle Of Los Angeles   

                 album_id                       artists  \
0  2eia0myWFgoHuttJytCxgX  ['Rage Against The Machine']   
1  2eia0myWFgoHuttJytCxgX  ['Rage Against The Machine']   
2  2eia0myWFgoHuttJytCxgX  ['Rage Against The Machine']   
3  2eia0myWFgoHuttJytCxgX  ['Rage Against The Machine']   
4  2eia0myWFgoHuttJytCxgX  ['Rage Against The Machine']   

                   artist_ids  track_number  disc_number  explicit  \
0  ['2d0hyoQ5ynDBnkvAbJKORj']             1           

,name,artists,album
791116,Citrus Light,"['Sin Fang', 'Sóley', 'Örvar Smárason']",Team Dreams
826612,The Sun Is Going Down II,['Sóley'],We Sink
443232,Green Eyes,['Steven Paul Ploog'],The Truth Is
416471,Trouble In Mind,['Jennifer Scott Quartet'],Live at the Cellar
485646,Meek; Wild,['Brooke Waggoner'],Go Easy Little Doves
